In [2]:
import matsim
import geopandas as gpd
import pandas as pd
import numpy as np
import os
import networkx as nx
import matplotlib.pyplot as plt
from haversine import haversine_vector, haversine, Unit

In [3]:
road_network_root = r'../../../../data/intermediate/test/freightEmissions/'
scenario_types = ['CB', 'Van', 'Basic']

# Test

In [14]:
scenario_type = 'CB'
network_type = 'car' if scenario_type in ['Van', 'Basic'] else 'bike'
network_filename = network_type + 'GemeenteLeuvenWithHbefaType.xml.gz'
road_network_path = os.path.join(road_network_root, network_filename)
road_network = matsim.read_network(road_network_path)

In [15]:
road_network.nodes

,x,y,node_id
0,175270.308015,173512.798102,1000630419
1,175160.942490,173472.188651,1000630421
2,175238.142000,173547.294517,1000630459
3,175338.370411,173481.813410,1000630462
4,175180.404081,173465.163020,1000630464
...,...,...,...
43358,171014.648515,180895.043949,splitNode_988732363_r_0_1
43359,171128.576126,180779.039304,splitNode_988732363_r_0_2
43360,174985.241824,172934.604012,splitNode_9904064_1_1
43361,174985.241824,172934.604012,splitNode_9904064_r_1_1


In [16]:
road_network.links

,length,freespeed,capacity,permlanes,oneway,modes,link_id,from_node,to_node
0,7.221720,13.888889,1277.777778,2.0,1,bike,100006229_1-1085376891_0,453075429,453075431
1,7.924752,13.888889,1277.777778,2.0,1,bike,100006230_1-1085376884_0,1472593946,1156261854
2,8.518773,13.888889,1277.777778,2.0,1,bike,100006231_0-1085376890_0,1156261856,453075428
3,50.128561,33.333333,2666.666667,3.5,1,bike,100010104_0,1156293852,1156293879
4,14.524426,33.333333,2666.666667,3.5,1,bike,100010104_1,1156293879,1156293875
...,...,...,...,...,...,...,...,...,...
96592,156.973685,33.333333,2666.666667,6.0,1,bike,splitLink_9904064_1_2,splitNode_9904064_1_1,2164607730
96593,156.973685,33.333333,2666.666667,6.0,1,bike,splitLink_9904064_r_1_1,2164607730,splitNode_9904064_r_1_1
96594,156.973685,33.333333,2666.666667,6.0,1,bike,splitLink_9904064_r_1_2,splitNode_9904064_r_1_1,80592606
96595,113.694402,13.888889,1277.777778,2.0,1,bike,splitLink_99346726_1_1,1149134729,splitNode_99346726_1_1


In [17]:
network_gdf = road_network.as_geo()
network_gdf

,length,freespeed,capacity,permlanes,oneway,modes,link_id,from_node,to_node,geometry
0,7.221720,13.888889,1277.777778,2.0,1,bike,100006229_1-1085376891_0,453075429,453075431,"LINESTRING (175094.462 171579.032, 175087.718 ..."
1,7.924752,13.888889,1277.777778,2.0,1,bike,100006230_1-1085376884_0,1472593946,1156261854,"LINESTRING (175094.956 171547.998, 175102.682 ..."
2,8.518773,13.888889,1277.777778,2.0,1,bike,100006231_0-1085376890_0,1156261856,453075428,"LINESTRING (175114.241 171571.774, 175106.042 ..."
3,50.128561,33.333333,2666.666667,3.5,1,bike,100010104_0,1156293852,1156293879,"LINESTRING (174065.630 177001.873, 174056.230 ..."
4,14.524426,33.333333,2666.666667,3.5,1,bike,100010104_1,1156293879,1156293875,"LINESTRING (174056.230 177051.111, 174070.620 ..."
...,...,...,...,...,...,...,...,...,...,...
96592,156.973685,33.333333,2666.666667,6.0,1,bike,splitLink_9904064_1_2,splitNode_9904064_1_1,2164607730,"LINESTRING (174985.242 172934.604, 174921.893 ..."
96593,156.973685,33.333333,2666.666667,6.0,1,bike,splitLink_9904064_r_1_1,2164607730,splitNode_9904064_r_1_1,"LINESTRING (174921.893 173078.221, 174985.242 ..."
96594,156.973685,33.333333,2666.666667,6.0,1,bike,splitLink_9904064_r_1_2,splitNode_9904064_r_1_1,80592606,"LINESTRING (174985.242 172934.604, 175048.590 ..."
96595,113.694402,13.888889,1277.777778,2.0,1,bike,splitLink_99346726_1_1,1149134729,splitNode_99346726_1_1,"LINESTRING (170932.314 173317.687, 171004.317 ..."


In [18]:
network_gdf.set_crs(epsg=31370, inplace=True)

,length,freespeed,capacity,permlanes,oneway,modes,link_id,from_node,to_node,geometry
0,7.221720,13.888889,1277.777778,2.0,1,bike,100006229_1-1085376891_0,453075429,453075431,"LINESTRING (175094.462 171579.032, 175087.718 ..."
1,7.924752,13.888889,1277.777778,2.0,1,bike,100006230_1-1085376884_0,1472593946,1156261854,"LINESTRING (175094.956 171547.998, 175102.682 ..."
2,8.518773,13.888889,1277.777778,2.0,1,bike,100006231_0-1085376890_0,1156261856,453075428,"LINESTRING (175114.241 171571.774, 175106.042 ..."
3,50.128561,33.333333,2666.666667,3.5,1,bike,100010104_0,1156293852,1156293879,"LINESTRING (174065.630 177001.873, 174056.230 ..."
4,14.524426,33.333333,2666.666667,3.5,1,bike,100010104_1,1156293879,1156293875,"LINESTRING (174056.230 177051.111, 174070.620 ..."
...,...,...,...,...,...,...,...,...,...,...
96592,156.973685,33.333333,2666.666667,6.0,1,bike,splitLink_9904064_1_2,splitNode_9904064_1_1,2164607730,"LINESTRING (174985.242 172934.604, 174921.893 ..."
96593,156.973685,33.333333,2666.666667,6.0,1,bike,splitLink_9904064_r_1_1,2164607730,splitNode_9904064_r_1_1,"LINESTRING (174921.893 173078.221, 174985.242 ..."
96594,156.973685,33.333333,2666.666667,6.0,1,bike,splitLink_9904064_r_1_2,splitNode_9904064_r_1_1,80592606,"LINESTRING (174985.242 172934.604, 175048.590 ..."
96595,113.694402,13.888889,1277.777778,2.0,1,bike,splitLink_99346726_1_1,1149134729,splitNode_99346726_1_1,"LINESTRING (170932.314 173317.687, 171004.317 ..."


## Convert gdf to networkx multi Di graph

In [7]:
def getNodeInfo(nodesDict,
                nodeCoords,
                ):
    nodesDictKeys = list(nodesDict.keys())
    nodesDictValues = list(nodesDict.values())

    node_id = None
    if nodesDict == {}:
        return node_id

    # If the node is in the nodeDict
    if nodeCoords in nodesDictValues:
        node_id = nodesDictKeys[nodesDictValues.index(nodeCoords)]

    else:  # Find the 'same' node in the nodeDict (distance < 0.2m)
        dist_array = haversine_vector(np.array(nodesDictValues),
                                      np.array([nodeCoords]),
                                      Unit.METERS,
                                      comb=True)
        # Get the index of the 'same' node (if applicable)
        try:
            closeNode_idx = np.where(dist_array < 0.2)[1][0]
        except:
            closeNode_idx = None

        # If the point is close to an existing node, return the node id of the existing node
        if closeNode_idx is not None:
            node_id = nodesDictKeys[closeNode_idx]

    return node_id

In [8]:
def create_graph(road_gdf: gpd.GeoDataFrame,
                 length_col='length',
                 
                 ):
    splitG = nx.DiGraph()

    # Create a dictionary to store the unique nodes (id, coords)
    nodesDict = {}

    start_nodes = []
    end_nodes = []

    node_idx = 0
    for idx, row in road_gdf.iterrows():
        # idx = row.index
        length = row[length_col]
        line = row['geometry']
        line_orig_id = row['link_id']

        # (lat, lon) format
        lineStartCoords = (line.coords[0][1], line.coords[0][0])
        lineEndCoords = (line.coords[-1][1], line.coords[-1][0])

        # Get the node id of the start and end nodes
        startNodeOrigId = row['from_node']
        endNodeOrigId = row['to_node']

        startNode_id = getNodeInfo(nodesDict, lineStartCoords)
        if startNode_id is None:
            startNode_id = node_idx
            node_idx += 1
            nodesDict[startNode_id] = lineStartCoords

        endNode_id = getNodeInfo(nodesDict, lineEndCoords)
        if endNode_id is None:
            endNode_id = node_idx
            node_idx += 1
            nodesDict[endNode_id] = lineEndCoords

        start_nodes.append(startNode_id)
        end_nodes.append(endNode_id)

        # Add nodes to the graph
        if not splitG.has_node(startNode_id):
            splitG.add_node(startNode_id, x=lineStartCoords[1], y=lineStartCoords[0], 
                            orig_id=startNodeOrigId # Add the original node id of the node
                            )
        if not splitG.has_node(endNode_id):
            splitG.add_node(endNode_id, x=lineEndCoords[1], y=lineEndCoords[0],
                            orig_id=endNodeOrigId # Add the original node id of the node
                            )
        splitG.add_edge(startNode_id, endNode_id,
                        length=length,
                        orig_id=line_orig_id,
                        idx=idx)

    splitG.graph['crs'] = "EPSG:4326"

    indexed_road = road_gdf.copy()

    indexed_road['start_node'] = start_nodes
    indexed_road['end_node'] = end_nodes

    return splitG, indexed_road

In [19]:
splitG, indexed_road = create_graph(network_gdf.to_crs('EPSG:4326'))

In [20]:
indexed_road = indexed_road.to_crs('EPSG:31370')
indexed_road


,length,freespeed,capacity,permlanes,oneway,modes,link_id,from_node,to_node,geometry,start_node,end_node
0,7.221720,13.888889,1277.777778,2.0,1,bike,100006229_1-1085376891_0,453075429,453075431,"LINESTRING (175094.461 171579.033, 175087.717 ...",0,1
1,7.924752,13.888889,1277.777778,2.0,1,bike,100006230_1-1085376884_0,1472593946,1156261854,"LINESTRING (175094.956 171547.998, 175102.682 ...",2,3
2,8.518773,13.888889,1277.777778,2.0,1,bike,100006231_0-1085376890_0,1156261856,453075428,"LINESTRING (175114.241 171571.775, 175106.042 ...",4,5
3,50.128561,33.333333,2666.666667,3.5,1,bike,100010104_0,1156293852,1156293879,"LINESTRING (174065.629 177001.873, 174056.229 ...",6,7
4,14.524426,33.333333,2666.666667,3.5,1,bike,100010104_1,1156293879,1156293875,"LINESTRING (174056.229 177051.111, 174070.619 ...",7,8
...,...,...,...,...,...,...,...,...,...,...,...,...
96592,156.973685,33.333333,2666.666667,6.0,1,bike,splitLink_9904064_1_2,splitNode_9904064_1_1,2164607730,"LINESTRING (174985.242 172934.604, 174921.893 ...",43158,42667
96593,156.973685,33.333333,2666.666667,6.0,1,bike,splitLink_9904064_r_1_1,2164607730,splitNode_9904064_r_1_1,"LINESTRING (174921.893 173078.221, 174985.242 ...",42667,43158
96594,156.973685,33.333333,2666.666667,6.0,1,bike,splitLink_9904064_r_1_2,splitNode_9904064_r_1_1,80592606,"LINESTRING (174985.242 172934.604, 175048.590 ...",43158,42666
96595,113.694402,13.888889,1277.777778,2.0,1,bike,splitLink_99346726_1_1,1149134729,splitNode_99346726_1_1,"LINESTRING (170932.314 173317.687, 171004.317 ...",42685,43159


## Calculate the centrality

### Degree centrality

In [21]:
graph_degree_centrality = nx.degree_centrality(splitG)
graph_degree_centrality

{0: 4.634027665145161e-05,
 1: 9.268055330290322e-05,
 2: 4.634027665145161e-05,
 3: 9.268055330290322e-05,
 4: 9.268055330290322e-05,
 5: 4.634027665145161e-05,
 6: 0.00013902082995435484,
 7: 9.268055330290322e-05,
 8: 4.634027665145161e-05,
 9: 6.951041497717742e-05,
 10: 0.00013902082995435484,
 11: 6.951041497717742e-05,
 12: 6.951041497717742e-05,
 13: 4.634027665145161e-05,
 14: 4.634027665145161e-05,
 15: 4.634027665145161e-05,
 16: 4.634027665145161e-05,
 17: 4.634027665145161e-05,
 18: 4.634027665145161e-05,
 19: 4.634027665145161e-05,
 20: 0.00013902082995435484,
 21: 4.634027665145161e-05,
 22: 4.634027665145161e-05,
 23: 4.634027665145161e-05,
 24: 4.634027665145161e-05,
 25: 4.634027665145161e-05,
 26: 4.634027665145161e-05,
 27: 4.634027665145161e-05,
 28: 4.634027665145161e-05,
 29: 4.634027665145161e-05,
 30: 4.634027665145161e-05,
 31: 4.634027665145161e-05,
 32: 9.268055330290322e-05,
 33: 0.00018536110660580645,
 34: 0.00011585069162862903,
 35: 9.268055330290322e-0

In [22]:
degree_centrality_df = pd.DataFrame(graph_degree_centrality.items(), columns=['node_id', 'degree_centrality'])
degree_centrality_df['orig_id'] = degree_centrality_df['node_id'].apply(lambda x: splitG.nodes[x]['orig_id'])
degree_centrality_df

,node_id,degree_centrality,orig_id
0,0,0.000046,453075429
1,1,0.000093,453075431
2,2,0.000046,1472593946
3,3,0.000093,1156261854
4,4,0.000093,1156261856
...,...,...,...
43155,43155,0.000093,splitNode_988732363_0_1
43156,43156,0.000093,splitNode_988732363_0_2
43157,43157,0.000046,246621219
43158,43158,0.000093,splitNode_9904064_1_1


In [23]:
degree_centrality_df.to_csv(f'../../../../data/clean/freightEmissions/networkCentrality/{network_type}Network/degreeCentrality.csv.gz')

### Betweeness centrality    

In [25]:
edges_betweeness = nx.edge_betweenness_centrality(G=splitG,
                                                  k=40000,
                                                normalized=True,
                                                weight='length'
                                                )
edges_betweeness

{(0, 1): 0.006180276324192195,
 (1, 5911): 0.008562104807146607,
 (1, 43117): 0.0022710901459892653,
 (2, 3): 0.0031143189071270636,
 (3, 658): 0.007107361015514308,
 (3, 3385): 0.0006154882045850633,
 (4, 5): 0.006176883477245518,
 (4, 3382): 0.0028091355453306794,
 (5, 0): 0.006178579900718856,
 (6, 7): 4.294528233328919e-05,
 (6, 42928): 0.00015028969866601633,
 (6, 42916): 0.00020848292907311438,
 (7, 8): 2.1473178009515906e-05,
 (7, 6): 4.633812927996637e-05,
 (8, 7): 2.3169601482854495e-05,
 (9, 10): 0.013481628195468612,
 (9, 8349): 0.00025759385178339525,
 (10, 11): 0.013625688369455952,
 (10, 25757): 1.9326343367148493e-08,
 (10, 9756): 4.294528233328919e-05,
 (11, 8353): 0.013766364822825426,
 (12, 13): 0.013720447578356566,
 (12, 8359): 4.3104187823196855e-05,
 (13, 25): 0.013722144001829905,
 (14, 15): 0.01370915240434421,
 (15, 16): 0.013710848827817548,
 (16, 17): 0.013712545251290887,
 (17, 18): 0.013714241674764226,
 (18, 19): 0.013715938098237564,
 (19, 20): 0.01371763

In [26]:
edges_betweeness_df = pd.DataFrame(edges_betweeness.items(), columns=['edge', 'betweeness'])
edges_betweeness_df['orig_id'] = edges_betweeness_df['edge'].apply(lambda x: splitG.edges.get(x)['orig_id'])
edges_betweeness_df

,edge,betweeness,orig_id
0,"(0, 1)",0.006180,100006229_1-1085376891_0
1,"(1, 5911)",0.008562,653482598_0
2,"(1, 43117)",0.002271,splitLink_653480981_r_0_1
3,"(2, 3)",0.003114,100006230_1-1085376884_0
4,"(3, 658)",0.007107,1085376882_0-231212154_0
...,...,...,...
96104,"(43156, 43155)",0.000046,splitLink_988732363_r_0_2
96105,"(43157, 43156)",0.000023,splitLink_988732363_r_0_1
96106,"(43158, 42667)",0.012783,splitLink_9904064_1_2
96107,"(43158, 42666)",0.015383,splitLink_9904064_r_1_2


In [27]:
edges_betweeness_df.to_csv(f'../../../../data/clean/freightEmissions/networkCentrality/{network_type}Network/edges_betweeness.csv.gz')

### Closeness centrality

In [28]:
closeness_centrality = nx.closeness_centrality(G=splitG,
                                               distance='length',
                                            )

In [29]:
closeness_centrality_df = pd.DataFrame(closeness_centrality.items(), columns=['node', 'closeness'])
closeness_centrality_df['orig_id'] = closeness_centrality_df['node'].apply(lambda x: splitG.nodes.get(x)['orig_id'])
closeness_centrality_df

,node,closeness,orig_id
0,0,0.000216,453075429
1,1,0.000219,453075431
2,2,0.000217,1472593946
3,3,0.000218,1156261854
4,4,0.000217,1156261856
...,...,...,...
43155,43155,0.000106,splitNode_988732363_0_1
43156,43156,0.000104,splitNode_988732363_0_2
43157,43157,0.000102,246621219
43158,43158,0.000273,splitNode_9904064_1_1


In [30]:
closeness_centrality_df.to_csv(f'../../../../data/clean/freightEmissions/networkCentrality/{network_type}Network/closeness_centrality.csv.gz')

### Eigenvector centrality

In [31]:
eigenvector_centrality = nx.eigenvector_centrality(G=splitG,
                                                   weight='length',
                                                   max_iter=3000
                                                )

In [32]:
eigenvector_centrality_df = pd.DataFrame(eigenvector_centrality.items(), columns=['node', 'eigenvector'])
eigenvector_centrality_df['orig_id'] = eigenvector_centrality_df['node'].apply(lambda x: splitG.nodes.get(x)['orig_id'])
eigenvector_centrality_df

,node,eigenvector,orig_id
0,0,1.798778e-90,453075429
1,1,5.424902e-76,453075431
2,2,1.190107e-81,1472593946
3,3,3.425355e-83,1156261854
4,4,1.287174e-87,1156261856
...,...,...,...
43155,43155,6.046168e-80,splitNode_988732363_0_1
43156,43156,5.639048e-80,splitNode_988732363_0_2
43157,43157,3.349820e-80,246621219
43158,43158,5.362966e-110,splitNode_9904064_1_1


In [33]:
eigenvector_centrality_df.to_csv(f'../../../../data/clean/freightEmissions/networkCentrality/{network_type}Network/eigenvector_centrality.csv.gz')

### Dispersion

In [ ]:
dispersion_centrality = nx.dispersion(splitG, u=0, v=1,normalized=True)

0.0

### Trophic levels

In [64]:
trophic_levels = nx.trophic_levels(splitG, weight='length')
trophic_levels

{0: -1.4896271269241523e+18,
 1: -1.4896271269241518e+18,
 2: -1.4896271269241518e+18,
 3: -1.489627126924011e+18,
 4: -1.4896271269240115e+18,
 5: -1.4896271269240115e+18,
 6: -1.4896271269240115e+18,
 7: -1.4896271269240115e+18,
 8: -1.4896271269240115e+18,
 9: -1.4896271269240773e+18,
 10: -1.4896271269240817e+18,
 11: -1.4896271269241034e+18,
 12: -1.4896271269240084e+18,
 13: -1.4896271269240102e+18,
 14: -1.4896271269241114e+18,
 15: -1.4896271269241088e+18,
 16: -1.4896271269241075e+18,
 17: -1.4896271269240927e+18,
 18: -1.4896271269240916e+18,
 19: -1.489627126924088e+18,
 20: -1.4896271269240845e+18,
 21: -1.4896271269240835e+18,
 22: -1.4896271269240822e+18,
 23: -1.489627126924107e+18,
 24: -1.4896271269241062e+18,
 25: -1.4896271269241057e+18,
 26: -1.4896271269241032e+18,
 27: -1.4896271269241004e+18,
 28: -1.4896271269240975e+18,
 29: -1.4896271269240955e+18,
 30: -1.4896271269240397e+18,
 31: -1.4896271269240387e+18,
 32: -1.4896271269240159e+18,
 33: -1.489627126924018

In [65]:
trophic_levels_df = pd.DataFrame(trophic_levels.items(), columns=['node', 'trophic_level'])
trophic_levels_df['orig_id'] = trophic_levels_df['node'].apply(lambda x: splitG.nodes.get(x)['orig_id'])
trophic_levels_df

,node,trophic_level,orig_id
0,0,-1.489627e+18,1156293852
1,1,-1.489627e+18,1156293879
2,2,-1.489627e+18,1156293875
3,3,-1.489627e+18,1156595483
4,4,-1.489627e+18,1156595495
...,...,...,...
27248,27248,-1.489627e+18,splitNode_9810850_3_1
27249,27249,-1.489627e+18,splitNode_98508213_11_1
27250,27250,-1.489627e+18,splitNode_9904064_1_1
27251,27251,-1.489627e+18,splitNode_9904390_5_1


### Network density

### Intersection density

### Depot number

### Depot Proximity